In [1]:
import time

import numpy as np
import pandas as pd
from pandas import DataFrame
import math

from matplotlib.pyplot import savefig

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
from torch.optim import AdamW
import random

import matplotlib.pyplot as plt
import scipy.sparse as sp

In [2]:
training_flow = pd.read_csv('data/training_flow.csv')
training_flow['Timestamp'] = pd.to_datetime(training_flow['Timestamp'])
training_flow.set_index('Timestamp', inplace=True)

training_speed = pd.read_csv('data/training_speed.csv')
training_speed['Timestamp'] = pd.to_datetime(training_speed['Timestamp'])
training_speed.set_index('Timestamp', inplace=True)

test_flow = pd.read_csv('data/test_flow.csv')
test_flow['Timestamp'] = pd.to_datetime(test_flow['Timestamp'])
test_flow.set_index('Timestamp', inplace=True)

test_speed = pd.read_csv('data/test_speed.csv')
test_speed['Timestamp'] = pd.to_datetime(test_speed['Timestamp'])
test_speed.set_index('Timestamp', inplace=True)

num_link = training_flow.shape[1]

In [3]:
time_leg = 24
prediction_horizon = 6

In [4]:
continuous_training_flow = []
continuous_training_speed = []

continuous_training_time = []

training_times = training_flow.index

for i in range(len(training_times) - time_leg - prediction_horizon + 1):
    
    # Check if the 30 time points starting from the current line are consecutive
    if all(training_times[i + j] == training_times[i] + pd.Timedelta(minutes=5 * j) for j in range(1,time_leg + prediction_horizon)):
        
        # If consecutive, extract the data from these 30 lines
        data_slice_flow = training_flow.iloc[i:i + time_leg + prediction_horizon].values
        data_slice_speed= training_speed.iloc[i:i + time_leg + prediction_horizon].values
        
        # Add the data to the list
        continuous_training_flow.append(data_slice_flow)
        continuous_training_speed.append(data_slice_speed)
        
        continuous_training_time.append(training_times[i])
        
flow_data_training = np.array(continuous_training_flow)
speed_data_training = np.array(continuous_training_speed)

flow_data_training.shape

(67752, 30, 150)

In [5]:
continuous_test_flow = []
continuous_test_speed = []

continuous_test_time = []

test_times = test_flow.index

for i in range(len(test_times) - time_leg - prediction_horizon + 1):
    
    # Check if the 30 time points starting from the current line are consecutive
    if all(test_times[i + j] == test_times[i] + pd.Timedelta(minutes=5 * j) for j in range(1,time_leg + prediction_horizon)):
        
        # If consecutive, extract the data from these 30 lines
        data_slice_flow = test_flow.iloc[i:i + time_leg + prediction_horizon].values
        data_slice_speed= test_speed.iloc[i:i + time_leg + prediction_horizon].values
        
        # Add the data to the list
        continuous_test_flow.append(data_slice_flow)
        continuous_test_speed.append(data_slice_speed)
        
        continuous_test_time.append(test_times[i])
        
flow_data_test = np.array(continuous_test_flow)
speed_data_test = np.array(continuous_test_speed)

flow_data_test.shape

(4891, 30, 150)

In [6]:
training_set_q = torch.from_numpy(flow_data_training).to(torch.float32)
test_set_q = torch.from_numpy(flow_data_test).to(torch.float32)

training_set_v = torch.from_numpy(speed_data_training).to(torch.float32)
test_set_v = torch.from_numpy(speed_data_test).to(torch.float32)

print(training_set_q.shape)
print(test_set_q.shape)
print(training_set_v.shape)
print(test_set_v.shape)

torch.Size([67752, 30, 150])
torch.Size([4891, 30, 150])
torch.Size([67752, 30, 150])
torch.Size([4891, 30, 150])


In [7]:
v_f = pd.read_csv('parameter/v_f.csv').values
k_c = pd.read_csv('parameter/k_c.csv').values
mm = pd.read_csv('parameter/mm.csv').values

v_f = torch.from_numpy(v_f).to(torch.float32).reshape(-1)
k_c = torch.from_numpy(k_c).to(torch.float32).reshape(-1)
mm = torch.from_numpy(mm).to(torch.float32).reshape(-1)

v_f.shape

torch.Size([150])

==============================   Prediction model   =================================

In [9]:
adj = pd.read_csv('data/adj_matrix/proximity_matrix.csv').values
adj_correlation_q = pd.read_csv('data/adj_matrix/correlation_matrix_flow.csv').values
adj_knn_q = pd.read_csv('data/adj_matrix/knn_matrix_flow.csv').values
adj_correlation_v = pd.read_csv('data/adj_matrix/correlation_matrix_speed.csv').values
adj_knn_v = pd.read_csv('data/adj_matrix/knn_matrix_speed.csv').values

In [10]:
adj_correlation_q = torch.from_numpy(adj_correlation_q).to(torch.float32)
adj_correlation_v = torch.from_numpy(adj_correlation_v).to(torch.float32)

In [11]:
# Adjacency matrix normalization
def adj_def(adj):
   
    adj = adj + sp.eye(adj.shape[0])
    rowsum = np.array(adj.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0
    r_mat_inv = sp.diags(r_inv)
    adj = r_mat_inv.dot(adj).A
    
    adj = torch.from_numpy(adj).to(torch.float32)

    return adj

In [12]:
adj_n = adj_def(adj)
print(adj_n.shape)
adj_knn_q = adj_def(adj_knn_q)
adj_knn_v = adj_def(adj_knn_v)

torch.Size([150, 150])


In [13]:
max_q = training_set_q.max(axis=0)[0].max(axis=0)[0]
min_q = training_set_q.min(axis=0)[0].min(axis=0)[0]

max_v = training_set_v.max(axis=0)[0].max(axis=0)[0]
min_v = training_set_v.min(axis=0)[0].min(axis=0)[0]

max_q.shape

torch.Size([150])

In [14]:
# Define the normalization function
def norm(x,max_x,min_x):
    x0 = 2*(x-min_x)/(max_x-min_x)-1
    return x0

In [15]:
# Define the inverse normalization function
def r_norm(x0,max_x,min_x):
    x = (x0+1) * (max_x-min_x)/2 + min_x
    return x

In [16]:
# Normalize flow and speed data
training_set_q0 = norm(training_set_q,max_q,min_q)
test_set_q0 = norm(test_set_q,max_q,min_q)

training_set_v0 = norm(training_set_v,max_v,min_v)
test_set_v0 = norm(test_set_v,max_v,min_v)

In [17]:
# Training set input and output
training_input_q = training_set_q0[:,:time_leg,:]
training_output_q = training_set_q0[:,time_leg:time_leg+prediction_horizon,:]

training_input_v = training_set_v0[:,:time_leg,:]
training_output_v = training_set_v0[:,time_leg:time_leg+prediction_horizon,:]

# Test set input and output
test_input_q = test_set_q0[:,:time_leg,:]
test_output_q = test_set_q0[:,time_leg:time_leg+prediction_horizon,:]

test_input_v = test_set_v0[:,:time_leg,:]
test_output_v = test_set_v0[:,time_leg:time_leg+prediction_horizon,:]

# Ground truth
training_true_q = training_set_q[:,time_leg:time_leg+prediction_horizon,:]
training_true_v = training_set_v[:,time_leg:time_leg+prediction_horizon,:]

test_true_q = test_set_q[:,time_leg:time_leg+prediction_horizon,:]
test_true_v = test_set_v[:,time_leg:time_leg+prediction_horizon,:]

In [19]:
num_link = training_set_q0.shape[2]
num_link

150

In [24]:
# Define the GraphConvolution layer
class GraphConvolution(Module):

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.matmul(input, self.weight)
        output = torch.matmul(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

# Define the LSTM unit
class LSTM(nn.Module):

    def __init__(self, input_size, hidden_size, return_sequence):
        super(LSTM, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.return_sequence = return_sequence
        #self.seq_length = seq_length
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)

    def forward(self, x):
        
        h_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        c_0 = Variable(torch.zeros(1, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        output, (h_n, c_n) = self.lstm(x, (h_0, c_0))
        if self.return_sequence==False:
            output = output[:,-1,:]
            output = output.contiguous().view(output.shape[0],-1)
        return output    

In [25]:
class GCN(Module):

    def __init__(self, nfea_gcn, nhid_gcn1, nhid_gcn2, nhid_gcn3, n_latent_gcn, num_link):
        super(GCN, self).__init__()
        
        self.nfea_gcn = nfea_gcn
        self.num_link = num_link
        
        self.nhid_gcn1 = nhid_gcn1
        self.nhid_gcn2 = nhid_gcn2
        self.nhid_gcn3 = nhid_gcn3
        
        self.gc11 = GraphConvolution(nfea_gcn, nhid_gcn1[0])
        self.gc12 = GraphConvolution(nhid_gcn1[0], nhid_gcn1[1])
        self.fc_n = nn.Linear(nhid_gcn1[2] * num_link, nhid_gcn1[2])
        
        
        self.gc21 = GraphConvolution(nfea_gcn, nhid_gcn2[0])
        self.gc22 = GraphConvolution(nhid_gcn2[0], nhid_gcn2[1])
        self.fc_m = nn.Linear(nhid_gcn2[2] * num_link, nhid_gcn2[2])
    
        self.gc31 = GraphConvolution(nfea_gcn, nhid_gcn3[0])
        self.gc32 = GraphConvolution(nhid_gcn3[0], nhid_gcn3[1])
        self.fc_s = nn.Linear(nhid_gcn3[2] * num_link, nhid_gcn3[2])
        
        self.fc = nn.Linear(nhid_gcn1[2], n_latent_gcn)
        
    def forward(self, x, adj_n, adj_c, adj_k):
        
        x1 = torch.tanh(self.gc11(x, adj_n))
        x1 = torch.tanh(self.gc12(x1, adj_n))
        x1 = x1.view(x1.shape[0],-1)
        x1 = torch.tanh(self.fc_n(x1))
        
        x2 = torch.tanh(self.gc21(x, adj_c))
        x2 = torch.tanh(self.gc22(x2, adj_c))
        x2 = x2.view(x2.shape[0],-1)
        x2 = torch.tanh(self.fc_m(x2))
        
        x3 = torch.tanh(self.gc31(x, adj_k))
        x3 = torch.tanh(self.gc32(x3, adj_k))
        x3 = x3.view(x3.shape[0],-1)
        x3 = torch.tanh(self.fc_s(x3))
        
        x0 = (x1 + x2 + x3)/3
        x0 = torch.tanh(self.fc(x0))
        
        return x0

In [32]:
class PIML(Module):
    
    def __init__(self, nfea_lstm, nfea_gcn, prediction_horizon, num_link,
                 nhid_lstm, nhid_gcn, n_latent_gcn, nhid_linear):
        super(PIML, self).__init__()
   
        self.nfea_lstm = nfea_lstm
        self.nfea_gcn = nfea_gcn
        self.prediction_horizon = prediction_horizon
        self.num_link = num_link
        self.nhid_linear = nhid_linear  

        # three-layer LSTM
        self.lstm1 = LSTM(nfea_lstm, nhid_lstm[0], return_sequence=True)
        self.lstm2 = LSTM(nhid_lstm[0], nhid_lstm[1], return_sequence=True)
        self.lstm3 = LSTM(nhid_lstm[1], nhid_lstm[2], return_sequence=False)
        
        # MGCN
        self.MGCN = GCN(nfea_gcn, nhid_gcn[0], nhid_gcn[1], nhid_gcn[2], n_latent_gcn, num_link)
        
        self.fc1 = nn.Linear(nhid_lstm[-1]+n_latent_gcn, nhid_linear[0])
        self.fc2 = nn.Linear(nhid_linear[0], prediction_horizon*nfea_lstm)
        
    def forward(self, fea, adj_n, adj_c, adj_k):
        
        x_t = self.lstm1(fea)
        x_t = self.lstm2(x_t)
        x_t = self.lstm3(x_t)

        x_s = fea.transpose(1,2)
        x_s = self.MGCN(x_s, adj_n, adj_c, adj_k)        

        x = torch.cat((x_t, x_s),dim=1)

        x = torch.tanh(self.fc1(x)) 
        x = torch.tanh(self.fc2(x))
        
        x = x.view(x.shape[0], self.prediction_horizon, self.nfea_lstm)

        return x

In [38]:
piml_q=PIML(nfea_lstm=num_link, nfea_gcn=time_leg, prediction_horizon=prediction_horizon,num_link = num_link,
            nhid_lstm=[256,256,256], nhid_gcn=[[32,32,32],[32,32,32],[32,32,32]], n_latent_gcn=512, nhid_linear=[512])

In [39]:
piml_v=PIML(nfea_lstm=num_link, nfea_gcn=time_leg, prediction_horizon=prediction_horizon,num_link = num_link,
            nhid_lstm=[256,256,256], nhid_gcn=[[32,32,32],[32,32,32],[32,32,32]], n_latent_gcn=512, nhid_linear=[512])

In [40]:
# Loss function
nn_loss = nn.MSELoss()
# Optimizers
optimizer = AdamW([
    {'params': piml_q.parameters(), 'lr': 0.001, 'weight_decay': 1e-2},
    {'params': piml_v.parameters(), 'lr': 0.001, 'weight_decay': 1e-2}
])
n_epochs = 2000
batch_size = 256
sample_size = training_set_q.shape[0]

In [41]:
def train(n_epochs, batch_size, sample_size):
    t0 = time.time()
    for epoch in range(n_epochs):
        
        idx = random.sample(range(0, sample_size),batch_size)
        
        input_q = training_input_q[idx]
        output_q = training_output_q[idx]
        
        input_v = training_input_v[idx]
        output_v = training_output_v[idx]
        
        optimizer.zero_grad()
        
        pred_q0 = piml_q(input_q, adj_n, adj_correlation_q, adj_knn_q)
        pred_v0 = piml_v(input_v, adj_n, adj_correlation_v, adj_knn_v)
        
        loss_q = nn_loss(output_q, pred_q0)
        loss_v = nn_loss(output_v, pred_v0)
        
        # S3
        pred_q = r_norm(pred_q0, max_q, min_q)
        pred_v = r_norm(pred_v0, max_v, min_v)
        v_cal = v_f / (1 + (pred_q/(pred_v*k_c))**mm) ** (2/mm)
        loss_fd_target = Variable(torch.Tensor(v_cal.shape).fill_(1.0), requires_grad=False)
        loss_fd = nn_loss(v_cal/pred_v, loss_fd_target)
        
        total_loss = loss_q + loss_v + 0.2 * loss_fd
            
        total_loss.backward()
        optimizer.step()
        
        print(
            "[Epoch %d/%d] [loss_q: %f] [loss_v: %f] [loss_fd: %f] [total_loss: %f] [time: %f]"
            % (epoch, n_epochs, loss_q.item(), loss_v.item(), loss_fd.item(), total_loss.item(), time.time()-t0)
        )

In [42]:
train(n_epochs = n_epochs, batch_size = batch_size, sample_size = sample_size)

[Epoch 0/2000] [loss_q: 0.218846] [loss_v: 0.332435] [loss_fd: 0.098412] [total_loss: 0.570964] [time: 2.304641]
[Epoch 1/2000] [loss_q: 0.197262] [loss_v: 0.312892] [loss_fd: 0.093555] [total_loss: 0.528865] [time: 4.372113]
[Epoch 2/2000] [loss_q: 0.169944] [loss_v: 0.235443] [loss_fd: 0.048844] [total_loss: 0.415156] [time: 6.453763]
[Epoch 3/2000] [loss_q: 0.126482] [loss_v: 0.169402] [loss_fd: 0.020569] [total_loss: 0.299997] [time: 8.334646]
[Epoch 4/2000] [loss_q: 0.097790] [loss_v: 0.175606] [loss_fd: 0.010359] [total_loss: 0.275467] [time: 10.301326]
[Epoch 5/2000] [loss_q: 0.085147] [loss_v: 0.147039] [loss_fd: 0.008378] [total_loss: 0.233861] [time: 11.882453]
[Epoch 6/2000] [loss_q: 0.074019] [loss_v: 0.143698] [loss_fd: 0.010973] [total_loss: 0.219912] [time: 13.104787]
[Epoch 7/2000] [loss_q: 0.069823] [loss_v: 0.137580] [loss_fd: 0.023337] [total_loss: 0.212070] [time: 14.131069]
[Epoch 8/2000] [loss_q: 0.060726] [loss_v: 0.118680] [loss_fd: 0.017595] [total_loss: 0.1829

[Epoch 72/2000] [loss_q: 0.023015] [loss_v: 0.047451] [loss_fd: 0.004723] [total_loss: 0.071412] [time: 65.485081]
[Epoch 73/2000] [loss_q: 0.022336] [loss_v: 0.047347] [loss_fd: 0.004867] [total_loss: 0.070657] [time: 66.257632]
[Epoch 74/2000] [loss_q: 0.021841] [loss_v: 0.047461] [loss_fd: 0.004717] [total_loss: 0.070245] [time: 67.032183]
[Epoch 75/2000] [loss_q: 0.022299] [loss_v: 0.046698] [loss_fd: 0.004728] [total_loss: 0.069943] [time: 67.789722]
[Epoch 76/2000] [loss_q: 0.022402] [loss_v: 0.047127] [loss_fd: 0.005130] [total_loss: 0.070555] [time: 68.564274]
[Epoch 77/2000] [loss_q: 0.022044] [loss_v: 0.047815] [loss_fd: 0.004720] [total_loss: 0.070803] [time: 69.363281]
[Epoch 78/2000] [loss_q: 0.020503] [loss_v: 0.042557] [loss_fd: 0.004415] [total_loss: 0.063943] [time: 70.158848]
[Epoch 79/2000] [loss_q: 0.022140] [loss_v: 0.047537] [loss_fd: 0.004608] [total_loss: 0.070598] [time: 70.901085]
[Epoch 80/2000] [loss_q: 0.023589] [loss_v: 0.047879] [loss_fd: 0.004918] [total

[Epoch 143/2000] [loss_q: 0.021235] [loss_v: 0.041246] [loss_fd: 0.004927] [total_loss: 0.063466] [time: 120.595840]
[Epoch 144/2000] [loss_q: 0.022662] [loss_v: 0.045488] [loss_fd: 0.005230] [total_loss: 0.069196] [time: 121.373393]
[Epoch 145/2000] [loss_q: 0.020880] [loss_v: 0.040208] [loss_fd: 0.004958] [total_loss: 0.062080] [time: 122.193977]
[Epoch 146/2000] [loss_q: 0.021704] [loss_v: 0.042516] [loss_fd: 0.004867] [total_loss: 0.065193] [time: 122.932503]
[Epoch 147/2000] [loss_q: 0.019850] [loss_v: 0.039467] [loss_fd: 0.004682] [total_loss: 0.060253] [time: 123.708055]
[Epoch 148/2000] [loss_q: 0.019705] [loss_v: 0.042278] [loss_fd: 0.004817] [total_loss: 0.062946] [time: 124.516630]
[Epoch 149/2000] [loss_q: 0.019947] [loss_v: 0.041769] [loss_fd: 0.004598] [total_loss: 0.062636] [time: 125.271167]
[Epoch 150/2000] [loss_q: 0.019859] [loss_v: 0.040051] [loss_fd: 0.004890] [total_loss: 0.060887] [time: 126.026706]
[Epoch 151/2000] [loss_q: 0.020514] [loss_v: 0.039117] [loss_fd:

[Epoch 214/2000] [loss_q: 0.018337] [loss_v: 0.032837] [loss_fd: 0.005095] [total_loss: 0.052193] [time: 175.846889]
[Epoch 215/2000] [loss_q: 0.018345] [loss_v: 0.034561] [loss_fd: 0.005213] [total_loss: 0.053948] [time: 176.696494]
[Epoch 216/2000] [loss_q: 0.017731] [loss_v: 0.034156] [loss_fd: 0.005582] [total_loss: 0.053003] [time: 177.466042]
[Epoch 217/2000] [loss_q: 0.017433] [loss_v: 0.034609] [loss_fd: 0.005182] [total_loss: 0.053078] [time: 178.289629]
[Epoch 218/2000] [loss_q: 0.018446] [loss_v: 0.035047] [loss_fd: 0.005483] [total_loss: 0.054589] [time: 179.047167]
[Epoch 219/2000] [loss_q: 0.018015] [loss_v: 0.034142] [loss_fd: 0.005172] [total_loss: 0.053191] [time: 179.781916]
[Epoch 220/2000] [loss_q: 0.017724] [loss_v: 0.035549] [loss_fd: 0.005101] [total_loss: 0.054293] [time: 180.611506]
[Epoch 221/2000] [loss_q: 0.017890] [loss_v: 0.035693] [loss_fd: 0.005638] [total_loss: 0.054710] [time: 181.389060]
[Epoch 222/2000] [loss_q: 0.017737] [loss_v: 0.034538] [loss_fd:

[Epoch 285/2000] [loss_q: 0.016901] [loss_v: 0.031454] [loss_fd: 0.005721] [total_loss: 0.049499] [time: 246.656916]
[Epoch 286/2000] [loss_q: 0.016561] [loss_v: 0.030578] [loss_fd: 0.005670] [total_loss: 0.048273] [time: 248.514240]
[Epoch 287/2000] [loss_q: 0.016088] [loss_v: 0.030805] [loss_fd: 0.005481] [total_loss: 0.047989] [time: 250.280498]
[Epoch 288/2000] [loss_q: 0.016906] [loss_v: 0.031626] [loss_fd: 0.005632] [total_loss: 0.049659] [time: 251.904654]
[Epoch 289/2000] [loss_q: 0.016494] [loss_v: 0.031011] [loss_fd: 0.005772] [total_loss: 0.048660] [time: 253.568836]
[Epoch 290/2000] [loss_q: 0.016073] [loss_v: 0.030593] [loss_fd: 0.005412] [total_loss: 0.047749] [time: 255.263551]
[Epoch 291/2000] [loss_q: 0.016927] [loss_v: 0.031270] [loss_fd: 0.005391] [total_loss: 0.049275] [time: 256.769622]
[Epoch 292/2000] [loss_q: 0.016348] [loss_v: 0.031392] [loss_fd: 0.005373] [total_loss: 0.048814] [time: 258.367764]
[Epoch 293/2000] [loss_q: 0.016314] [loss_v: 0.031284] [loss_fd:

...

[Epoch 1682/2000] [loss_q: 0.009399] [loss_v: 0.013619] [loss_fd: 0.006979] [total_loss: 0.024414] [time: 2241.943086]
[Epoch 1683/2000] [loss_q: 0.009836] [loss_v: 0.014317] [loss_fd: 0.007400] [total_loss: 0.025634] [time: 2243.525215]
[Epoch 1684/2000] [loss_q: 0.009630] [loss_v: 0.014919] [loss_fd: 0.007014] [total_loss: 0.025952] [time: 2245.345510]
[Epoch 1685/2000] [loss_q: 0.009545] [loss_v: 0.014176] [loss_fd: 0.007044] [total_loss: 0.025130] [time: 2247.086747]
[Epoch 1686/2000] [loss_q: 0.010283] [loss_v: 0.014699] [loss_fd: 0.007461] [total_loss: 0.026475] [time: 2248.835994]
[Epoch 1687/2000] [loss_q: 0.009429] [loss_v: 0.013831] [loss_fd: 0.006925] [total_loss: 0.024645] [time: 2250.696320]
[Epoch 1688/2000] [loss_q: 0.009651] [loss_v: 0.014262] [loss_fd: 0.007128] [total_loss: 0.025339] [time: 2252.587664]
[Epoch 1689/2000] [loss_q: 0.009386] [loss_v: 0.014102] [loss_fd: 0.006702] [total_loss: 0.024828] [time: 2254.487015]
[Epoch 1690/2000] [loss_q: 0.009840] [loss_v: 0.

[Epoch 1751/2000] [loss_q: 0.009547] [loss_v: 0.014353] [loss_fd: 0.007308] [total_loss: 0.025361] [time: 2359.702956]
[Epoch 1752/2000] [loss_q: 0.009632] [loss_v: 0.014573] [loss_fd: 0.007554] [total_loss: 0.025716] [time: 2361.355130]
[Epoch 1753/2000] [loss_q: 0.009343] [loss_v: 0.013867] [loss_fd: 0.007414] [total_loss: 0.024693] [time: 2363.104405]
[Epoch 1754/2000] [loss_q: 0.009150] [loss_v: 0.014419] [loss_fd: 0.006881] [total_loss: 0.024946] [time: 2364.797580]
[Epoch 1755/2000] [loss_q: 0.009465] [loss_v: 0.013748] [loss_fd: 0.006934] [total_loss: 0.024600] [time: 2366.523812]
[Epoch 1756/2000] [loss_q: 0.009383] [loss_v: 0.014238] [loss_fd: 0.006778] [total_loss: 0.024976] [time: 2368.479227]
[Epoch 1757/2000] [loss_q: 0.009519] [loss_v: 0.014257] [loss_fd: 0.007375] [total_loss: 0.025251] [time: 2370.335524]
[Epoch 1758/2000] [loss_q: 0.009560] [loss_v: 0.014228] [loss_fd: 0.007141] [total_loss: 0.025216] [time: 2372.003279]
[Epoch 1759/2000] [loss_q: 0.009333] [loss_v: 0.

[Epoch 1820/2000] [loss_q: 0.010081] [loss_v: 0.015026] [loss_fd: 0.007663] [total_loss: 0.026640] [time: 2475.456562]
[Epoch 1821/2000] [loss_q: 0.009248] [loss_v: 0.013960] [loss_fd: 0.007308] [total_loss: 0.024670] [time: 2477.019674]
[Epoch 1822/2000] [loss_q: 0.008940] [loss_v: 0.013028] [loss_fd: 0.006803] [total_loss: 0.023328] [time: 2478.607807]
[Epoch 1823/2000] [loss_q: 0.009363] [loss_v: 0.013808] [loss_fd: 0.006952] [total_loss: 0.024562] [time: 2480.407085]
[Epoch 1824/2000] [loss_q: 0.009507] [loss_v: 0.013991] [loss_fd: 0.007347] [total_loss: 0.024967] [time: 2482.129309]
[Epoch 1825/2000] [loss_q: 0.009479] [loss_v: 0.013640] [loss_fd: 0.006661] [total_loss: 0.024451] [time: 2483.842276]
[Epoch 1826/2000] [loss_q: 0.009791] [loss_v: 0.014301] [loss_fd: 0.007389] [total_loss: 0.025570] [time: 2485.501457]
[Epoch 1827/2000] [loss_q: 0.009267] [loss_v: 0.013107] [loss_fd: 0.006867] [total_loss: 0.023748] [time: 2487.249704]
[Epoch 1828/2000] [loss_q: 0.009555] [loss_v: 0.

[Epoch 1889/2000] [loss_q: 0.009355] [loss_v: 0.013583] [loss_fd: 0.007415] [total_loss: 0.024421] [time: 2582.655919]
[Epoch 1890/2000] [loss_q: 0.009305] [loss_v: 0.013670] [loss_fd: 0.006945] [total_loss: 0.024364] [time: 2583.408454]
[Epoch 1891/2000] [loss_q: 0.009613] [loss_v: 0.014286] [loss_fd: 0.007459] [total_loss: 0.025391] [time: 2584.185007]
[Epoch 1892/2000] [loss_q: 0.008887] [loss_v: 0.013454] [loss_fd: 0.007091] [total_loss: 0.023759] [time: 2584.980574]
[Epoch 1893/2000] [loss_q: 0.009390] [loss_v: 0.013521] [loss_fd: 0.006936] [total_loss: 0.024299] [time: 2585.745118]
[Epoch 1894/2000] [loss_q: 0.009119] [loss_v: 0.013652] [loss_fd: 0.007110] [total_loss: 0.024193] [time: 2586.502657]
[Epoch 1895/2000] [loss_q: 0.009832] [loss_v: 0.014052] [loss_fd: 0.006987] [total_loss: 0.025282] [time: 2587.290218]
[Epoch 1896/2000] [loss_q: 0.009032] [loss_v: 0.013009] [loss_fd: 0.007290] [total_loss: 0.023499] [time: 2588.044032]
[Epoch 1897/2000] [loss_q: 0.009341] [loss_v: 0.

[Epoch 1958/2000] [loss_q: 0.009371] [loss_v: 0.014424] [loss_fd: 0.007325] [total_loss: 0.025260] [time: 2635.932638]
[Epoch 1959/2000] [loss_q: 0.009629] [loss_v: 0.013366] [loss_fd: 0.007073] [total_loss: 0.024409] [time: 2636.706189]
[Epoch 1960/2000] [loss_q: 0.009291] [loss_v: 0.013586] [loss_fd: 0.007223] [total_loss: 0.024322] [time: 2637.471898]
[Epoch 1961/2000] [loss_q: 0.009731] [loss_v: 0.014080] [loss_fd: 0.007520] [total_loss: 0.025315] [time: 2638.243447]
[Epoch 1962/2000] [loss_q: 0.009395] [loss_v: 0.013691] [loss_fd: 0.006758] [total_loss: 0.024438] [time: 2639.012995]
[Epoch 1963/2000] [loss_q: 0.009296] [loss_v: 0.013508] [loss_fd: 0.007022] [total_loss: 0.024208] [time: 2639.768532]
[Epoch 1964/2000] [loss_q: 0.009506] [loss_v: 0.014215] [loss_fd: 0.007196] [total_loss: 0.025160] [time: 2640.555092]
[Epoch 1965/2000] [loss_q: 0.009510] [loss_v: 0.013900] [loss_fd: 0.007185] [total_loss: 0.024846] [time: 2641.289615]
[Epoch 1966/2000] [loss_q: 0.009705] [loss_v: 0.

In [43]:
test_pred_q0 = piml_q(test_input_q, adj_n, adj_correlation_q, adj_knn_q)
test_pred_q = r_norm(test_pred_q0, max_q, min_q)

test_pred_v0 = piml_v(test_input_v, adj_n, adj_correlation_v, adj_knn_v)
test_pred_v = r_norm(test_pred_v0, max_v, min_v)

In [44]:
# 0.2
print(nn_loss(test_pred_q, test_true_q)**0.5)
print(nn_loss(test_pred_v, test_true_v)**0.5)

l1_loss = nn.L1Loss()
print(l1_loss(test_pred_q, test_true_q))
print(l1_loss(test_pred_v, test_true_v))

mape_q = torch.abs(test_pred_q-test_true_q)/test_true_q
mape_q = mape_q[mape_q<1000]

mape_v = torch.abs(test_pred_v-test_true_v)/test_true_v
mape_v = mape_v[mape_v<1000]

print(torch.mean(mape_q))
print(torch.mean(mape_v))

tensor(112.8850, grad_fn=<PowBackward0>)
tensor(5.1594, grad_fn=<PowBackward0>)
tensor(80.5469, grad_fn=<MeanBackward0>)
tensor(3.3322, grad_fn=<MeanBackward0>)
tensor(0.0944, grad_fn=<MeanBackward0>)
tensor(0.0801, grad_fn=<MeanBackward0>)
